In [7]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from collections import deque
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from xgboost import cv
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer

# Divided


## XGBoost


In [2]:
minus = pd.read_csv('minus_data_for_ml2.csv', index_col=0)
plus = pd.read_csv('data_for_ml2.csv', index_col=0)
train = pd.read_csv('train.csv', index_col=0)
minus.loc[train.index, 'class'] = train['class']
plus.loc[train.index, 'class'] = train['class']
plus['D_mean'] = plus.apply(lambda x: x.D_sum / x.D_count, axis=1)
plus['K_mean'] = plus.apply(lambda x: x.K_sum / x.K_count, axis=1)
minus['D_mean'] = minus.apply(lambda x: x.D_sum / x.D_count, axis=1)
minus['K_mean'] = minus.apply(lambda x: x.K_sum / x.K_count, axis=1)

/tmp/ipykernel_67521/3359218865.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  minus.loc[train.index, 'class'] = train['class']
/tmp/ipykernel_67521/3359218865.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  plus.loc[train.index, 'class'] = train['class']


In [3]:
plus

,class,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,,
0,D,777.614282,6911.0,0.112518,189.940639,3088.0,0.061509
1,K,379.111613,6912.0,0.054848,320.586678,3087.0,0.103851
2,D,464.316126,6911.0,0.067185,156.871831,3088.0,0.050800
3,K,294.214732,6912.0,0.042566,211.299259,3087.0,0.068448
4,D,820.776927,6911.0,0.118764,196.406639,3088.0,0.063603
...,...,...,...,...,...,...,...
14995,NaN,892.810953,6912.0,0.129168,194.093758,3088.0,0.062854
14996,NaN,307.139350,6912.0,0.044436,273.660614,3088.0,0.088621
14997,NaN,367.227889,6912.0,0.053129,219.240044,3088.0,0.070997


In [4]:
minus

,class,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,,
0,D,-405.776287,6911.0,-0.058715,-367.086575,3088.0,-0.118875
1,K,-665.306470,6912.0,-0.096254,-164.958415,3087.0,-0.053436
2,D,-349.802807,6911.0,-0.050615,-226.382758,3088.0,-0.073310
3,K,-451.004127,6912.0,-0.065249,-135.502683,3087.0,-0.043895
4,D,-445.659327,6911.0,-0.064486,-381.058482,3088.0,-0.123400
...,...,...,...,...,...,...,...
14995,NaN,-436.845293,6912.0,-0.063201,-412.016401,3088.0,-0.133425
14996,NaN,-552.881760,6912.0,-0.079989,-137.054831,3088.0,-0.044383
14997,NaN,-487.924329,6912.0,-0.070591,-171.393176,3088.0,-0.055503


In [6]:
train.value_counts()

class
D        6912
K        3088
Name: count, dtype: int64

### Plus


In [6]:
X, y = plus.loc[0:9999].drop(columns=['class']), plus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = plus.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1212)
X_train

/tmp/ipykernel_66758/310420619.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = plus.loc[0:9999].drop(columns=['class']), plus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,
6924,733.571073,6911.0,0.106145,168.244496,3088.0,0.054483
3218,258.404984,6911.0,0.037390,160.409217,3088.0,0.051946
6505,350.642826,6911.0,0.050737,295.814766,3088.0,0.095795
6914,576.206706,6911.0,0.083375,132.365749,3088.0,0.042865
5450,747.772729,6912.0,0.108185,179.604189,3087.0,0.058181
...,...,...,...,...,...,...
1125,396.871263,6912.0,0.057418,358.504243,3087.0,0.116134
2626,839.154667,6911.0,0.121423,193.694831,3088.0,0.062725
7606,420.633505,6911.0,0.060864,153.079281,3088.0,0.049572


In [7]:
X_val

,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,
551,792.994341,6911.0,0.114744,202.865289,3088.0,0.065695
2743,295.676851,6911.0,0.042784,156.975110,3088.0,0.050834
7051,379.128083,6911.0,0.054859,131.588459,3088.0,0.042613
7945,652.817843,6911.0,0.094461,209.896134,3088.0,0.067972
7976,496.583288,6912.0,0.071844,478.868471,3087.0,0.155124
...,...,...,...,...,...,...
2158,736.670664,6911.0,0.106594,175.671996,3088.0,0.056889
1982,402.772199,6912.0,0.058271,369.456179,3087.0,0.119681
1871,608.053160,6911.0,0.087983,171.750164,3088.0,0.055619


In [8]:
y_val

id
551     0
2743    0
7051    0
7945    0
7976    1
       ..
2158    0
1982    1
1871    0
3992    1
9179    0
Name: class, Length: 2000, dtype: int64

In [9]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 5, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61426
[1]	validation_0-logloss:0.60457
[2]	validation_0-logloss:0.59513
[3]	validation_0-logloss:0.58592
[4]	validation_0-logloss:0.57694
[5]	validation_0-logloss:0.56817
[6]	validation_0-logloss:0.55961
[7]	validation_0-logloss:0.55125
[8]	validation_0-logloss:0.54307
[9]	validation_0-logloss:0.53508
[10]	validation_0-logloss:0.52726
[11]	validation_0-logloss:0.51962
[12]	validation_0-logloss:0.51213
[13]	validation_0-logloss:0.50481
[14]	validation_0-logloss:0.49763
[15]	validation_0-logloss:0.49061
[16]	validation_0-logloss:0.48372
[17]	validation_0-logloss:0.47697
[18]	validation_0-logloss:0.47036
[19]	validation_0-logloss:0.46387
[20]	validation_0-logloss:0.45751
[21]	validation_0-logloss:0.45127
[22]	validation_0-logloss:0.44515
[23]	validation_0-logloss:0.43915
[24]	validation_0-logloss:0.43325
[25]	validation_0-logloss:0.42746
[26]	validation_0-logloss:0.42177
[27]	validation_0-logloss:0.41619
[28]	validation_0-logloss:0.41071
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [10]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 1.000000000000


In [11]:
y_val.value_counts()

class
0    1367
1     633
Name: count, dtype: int64

In [12]:
pd.Series(clf.predict(X_val)).value_counts()

0    1367
1     633
Name: count, dtype: int64

In [13]:
clf.predict(X_test.drop('class', axis=1))

array([1, 1, 1, ..., 1, 1, 1])

In [14]:
pred_classes = pd.DataFrame({'id': np.arange(10000, 15000),
                'class': [np.nan] * 5000})
pred_classes

,id,class
0,10000,NaN
1,10001,NaN
2,10002,NaN
3,10003,NaN
4,10004,NaN
...,...,...
4995,14995,NaN
4996,14996,NaN
4997,14997,NaN
4998,14998,NaN


In [15]:
pred_classes['class'] = clf.predict_proba(X_test.drop('class', axis=1))[:, 1]

In [16]:
pred_classes['class'].value_counts()

class
0.995019    5000
Name: count, dtype: int64

In [22]:
clf.feature_importances_

array([0., 1., 0., 0., 0., 0.], dtype=float32)

### Minus


In [61]:
X, y = minus.loc[0:9999].drop(columns=['class']), minus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = minus.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1212)
X_train

/tmp/ipykernel_62673/388348891.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = minus.loc[0:9999].drop(columns=['class']), minus.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,
6924,-383.983402,2763.000000,-0.138973,-346.120014,1346.833333,-0.256988
3218,-364.500352,2897.000000,-0.125820,-127.461724,1250.000000,-0.101969
6505,-633.585025,2896.333333,-0.218754,-153.978012,1226.166667,-0.125577
6914,-297.821153,2780.666667,-0.107104,-270.438693,1343.833333,-0.201244
5450,-417.377456,2817.500000,-0.148138,-317.997614,1330.500000,-0.239006
...,...,...,...,...,...,...
1125,-789.833391,2882.833333,-0.273978,-189.426389,1209.500000,-0.156615
2626,-424.327595,2778.500000,-0.152718,-375.366193,1355.000000,-0.277023
7606,-337.197303,2879.500000,-0.117103,-196.913304,1329.000000,-0.148167


In [62]:
X_val

,D_sum,D_count,D_mean,K_sum,K_count,K_mean
id,,,,,,
551,-449.596535,2810.500000,-0.159970,-362.984396,1367.166667,-0.265501
2743,-359.419068,2903.000000,-0.123810,-138.537018,1254.666667,-0.110417
7051,-272.502854,2757.000000,-0.098840,-175.953896,1312.333333,-0.134077
7945,-461.242724,2834.333333,-0.162734,-317.696880,1360.666667,-0.233486
7976,-999.827115,2864.166667,-0.349081,-223.262296,1176.500000,-0.189768
...,...,...,...,...,...,...
2158,-387.150528,2757.666667,-0.140391,-357.323985,1336.166667,-0.267425
1982,-791.727115,2896.166667,-0.273371,-182.451662,1209.333333,-0.150870
1871,-382.533684,2775.666667,-0.137817,-272.766367,1304.833333,-0.209043


In [63]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 5, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61412	validation_1-logloss:0.62175
[1]	validation_0-logloss:0.61154	validation_1-logloss:0.61934
[2]	validation_0-logloss:0.60901	validation_1-logloss:0.61699
[3]	validation_0-logloss:0.60654	validation_1-logloss:0.61472
[4]	validation_0-logloss:0.60413	validation_1-logloss:0.61247
[5]	validation_0-logloss:0.60176	validation_1-logloss:0.61029
[6]	validation_0-logloss:0.59943	validation_1-logloss:0.60809
[7]	validation_0-logloss:0.59715	validation_1-logloss:0.60594
[8]	validation_0-logloss:0.59492	validation_1-logloss:0.60384
[9]	validation_0-logloss:0.59273	validation_1-logloss:0.60177
[10]	validation_0-logloss:0.59059	validation_1-logloss:0.59975
[11]	validation_0-logloss:0.58849	validation_1-logloss:0.59780
[12]	validation_0-logloss:0.58641	validation_1-logloss:0.59587
[13]	validation_0-logloss:0.58438	validation_1-logloss:0.59401
[14]	validation_0-logloss:0.58241	validation_1-logloss:0.59215
[15]	validation_0-logloss:0.58045	validation_1-logloss:0.59037
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [64]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 0.772500000000


### Both


In [72]:
minus = minus.rename({'D_sum': 'minus_D_sum', 'D_count': 'minus_D_count', 'D_mean': 'minus_D_mean', 'K_sum': 'minus_K_sum', 'K_count': 'minus_K_count', 'K_mean': 'minus_K_mean'}, axis=1)
minus.reset_index()

,id,class,minus_D_sum,minus_D_count,minus_D_mean,minus_K_sum,minus_K_count,minus_K_mean
0,0,D,-405.776287,2756.500000,-0.147207,-367.086575,1339.000000,-0.274150
1,1,K,-665.306470,2913.833333,-0.228327,-164.958415,1236.166667,-0.133444
2,2,D,-349.802807,2779.833333,-0.125836,-226.382758,1319.666667,-0.171545
3,3,K,-451.004127,2890.333333,-0.156039,-135.502683,1256.000000,-0.107884
4,4,D,-445.659327,2794.000000,-0.159506,-381.058482,1352.333333,-0.281779
...,...,...,...,...,...,...,...,...
14995,14995,NaN,-436.845293,2759.333333,-0.158316,-412.016401,1344.333333,-0.306484
14996,14996,NaN,-552.881760,2931.500000,-0.188600,-137.054831,1231.166667,-0.111321
14997,14997,NaN,-487.924329,2918.500000,-0.167183,-171.393176,1261.166667,-0.135900
14998,14998,NaN,-455.885051,2907.833333,-0.156778,-214.564370,1316.666667,-0.162960


In [73]:
plus.reset_index()

,id,class,D_sum,D_count,D_mean,K_sum,K_count,K_mean
0,0,D,777.614282,2975.333333,0.261354,189.940639,1218.666667,0.155859
1,1,K,379.111613,2749.833333,0.137867,320.586678,1293.166667,0.247908
2,2,D,464.316126,2942.166667,0.157814,156.871831,1235.500000,0.126970
3,3,K,294.214732,2791.833333,0.105384,211.299259,1292.666667,0.163460
4,4,D,820.776927,3009.000000,0.272774,196.406639,1249.500000,0.157188
...,...,...,...,...,...,...,...,...
14995,14995,NaN,892.810953,3019.000000,0.295731,194.093758,1227.333333,0.158143
14996,14996,NaN,307.139350,2754.333333,0.111511,273.660614,1323.500000,0.206770
14997,14997,NaN,367.227889,2820.833333,0.130184,219.240044,1303.333333,0.168215
14998,14998,NaN,471.464040,2966.333333,0.158938,215.806324,1298.000000,0.166261


In [77]:
Full = pd.merge(plus.reset_index(), minus.reset_index(), how='inner', on=['id', 'class']).set_index('id')
Full

,class,D_sum,D_count,D_mean,K_sum,K_count,K_mean,minus_D_sum,minus_D_count,minus_D_mean,minus_K_sum,minus_K_count,minus_K_mean
id,,,,,,,,,,,,,
0,D,777.614282,2975.333333,0.261354,189.940639,1218.666667,0.155859,-405.776287,2756.500000,-0.147207,-367.086575,1339.000000,-0.274150
1,K,379.111613,2749.833333,0.137867,320.586678,1293.166667,0.247908,-665.306470,2913.833333,-0.228327,-164.958415,1236.166667,-0.133444
2,D,464.316126,2942.166667,0.157814,156.871831,1235.500000,0.126970,-349.802807,2779.833333,-0.125836,-226.382758,1319.666667,-0.171545
3,K,294.214732,2791.833333,0.105384,211.299259,1292.666667,0.163460,-451.004127,2890.333333,-0.156039,-135.502683,1256.000000,-0.107884
4,D,820.776927,3009.000000,0.272774,196.406639,1249.500000,0.157188,-445.659327,2794.000000,-0.159506,-381.058482,1352.333333,-0.281779
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,NaN,892.810953,3019.000000,0.295731,194.093758,1227.333333,0.158143,-436.845293,2759.333333,-0.158316,-412.016401,1344.333333,-0.306484
14996,NaN,307.139350,2754.333333,0.111511,273.660614,1323.500000,0.206770,-552.881760,2931.500000,-0.188600,-137.054831,1231.166667,-0.111321
14997,NaN,367.227889,2820.833333,0.130184,219.240044,1303.333333,0.168215,-487.924329,2918.500000,-0.167183,-171.393176,1261.166667,-0.135900


In [80]:
X, y = Full.loc[0:9999].drop(columns=['class']), Full.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = Full.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=47)
X_train

/tmp/ipykernel_62673/3470367663.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = Full.loc[0:9999].drop(columns=['class']), Full.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


,D_sum,D_count,D_mean,K_sum,K_count,K_mean,minus_D_sum,minus_D_count,minus_D_mean,minus_K_sum,minus_K_count,minus_K_mean
id,,,,,,,,,,,,
1402,647.190639,2956.833333,0.218880,175.884530,1254.000000,0.140259,-399.753331,2807.666667,-0.142379,-282.637091,1328.333333,-0.212776
954,674.122952,2994.333333,0.225133,171.121413,1229.166667,0.139217,-384.129908,2777.166667,-0.138317,-299.793299,1342.166667,-0.223365
6196,896.093021,3043.666667,0.294412,192.294736,1234.500000,0.155767,-437.556036,2783.000000,-0.157225,-413.445182,1368.333333,-0.302152
6331,296.203553,2706.166667,0.109455,212.818351,1264.166667,0.168347,-458.153129,2836.833333,-0.161502,-128.616960,1214.500000,-0.105901
7124,836.217539,3015.666667,0.277291,187.016247,1227.166667,0.152397,-418.301584,2758.666667,-0.151632,-381.965775,1355.666667,-0.281755
...,...,...,...,...,...,...,...,...,...,...,...,...
2896,674.756683,2947.666667,0.228912,163.138617,1209.666667,0.134862,-338.922715,2736.833333,-0.123838,-310.058906,1311.166667,-0.236476
8883,408.535379,2876.333333,0.142033,169.029254,1264.666667,0.133655,-360.218678,2834.166667,-0.127099,-184.617924,1287.500000,-0.143393
6728,374.358749,2825.500000,0.132493,253.623731,1313.166667,0.193139,-531.042934,2937.166667,-0.180801,-158.397385,1269.166667,-0.124804


In [81]:
X_val

,D_sum,D_count,D_mean,K_sum,K_count,K_mean,minus_D_sum,minus_D_count,minus_D_mean,minus_K_sum,minus_K_count,minus_K_mean
id,,,,,,,,,,,,
7432,711.130310,3002.666667,0.236833,168.514607,1226.666667,0.137376,-373.145022,2781.166667,-0.134169,-337.090598,1349.833333,-0.249728
5444,822.733848,3040.333333,0.270606,190.259241,1248.500000,0.152390,-427.408808,2803.333333,-0.152464,-379.591284,1359.166667,-0.279282
5448,637.147331,2968.500000,0.214636,173.200377,1233.666667,0.140395,-373.634929,2769.333333,-0.134919,-285.319944,1334.166667,-0.213856
8654,379.423348,2738.166667,0.138568,264.826107,1276.166667,0.207517,-560.281246,2866.666667,-0.195447,-175.887113,1234.500000,-0.142476
7543,696.028596,2962.833333,0.234920,176.748397,1229.000000,0.143815,-390.389425,2765.666667,-0.141156,-323.843411,1339.166667,-0.241825
...,...,...,...,...,...,...,...,...,...,...,...,...
6746,977.572108,3012.000000,0.324559,211.446100,1220.166667,0.173293,-471.239149,2760.166667,-0.170729,-456.940194,1346.833333,-0.339270
2903,559.679999,2948.166667,0.189840,156.055630,1258.166667,0.124034,-358.548981,2819.000000,-0.127190,-257.623676,1322.000000,-0.194874
3893,647.267784,2973.166667,0.217703,172.695380,1226.000000,0.140861,-385.834148,2760.833333,-0.139753,-304.092706,1338.833333,-0.227133


In [83]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 5, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61524	validation_1-logloss:0.61725
[1]	validation_0-logloss:0.61259	validation_1-logloss:0.61491
[2]	validation_0-logloss:0.60998	validation_1-logloss:0.61267
[3]	validation_0-logloss:0.60745	validation_1-logloss:0.61043
[4]	validation_0-logloss:0.60497	validation_1-logloss:0.60825
[5]	validation_0-logloss:0.60253	validation_1-logloss:0.60615
[6]	validation_0-logloss:0.60015	validation_1-logloss:0.60409
[7]	validation_0-logloss:0.59782	validation_1-logloss:0.60206
[8]	validation_0-logloss:0.59552	validation_1-logloss:0.60008
[9]	validation_0-logloss:0.59327	validation_1-logloss:0.59816
[10]	validation_0-logloss:0.59105	validation_1-logloss:0.59630
[11]	validation_0-logloss:0.58889	validation_1-logloss:0.59444
[12]	validation_0-logloss:0.58677	validation_1-logloss:0.59264
[13]	validation_0-logloss:0.58471	validation_1-logloss:0.59083
[14]	validation_0-logloss:0.58267	validation_1-logloss:0.58908
[15]	validation_0-logloss:0.58068	validation_1-logloss:0.58738
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [84]:
test_accuracy = accuracy_score(y_val, clf.predict(X_val))
print(f"Test accuracy: {test_accuracy:.12f}")

Test accuracy: 0.775500000000


In [99]:
pred_classes = pd.DataFrame({'id': np.arange(10000, 15000),
                'class': [np.nan] * 5000})
pred_classes

,id,class
0,10000,NaN
1,10001,NaN
2,10002,NaN
3,10003,NaN
4,10004,NaN
...,...,...
4995,14995,NaN
4996,14996,NaN
4997,14997,NaN
4998,14998,NaN


In [107]:
pred_classes['class'] = clf.predict(X_test.drop('class', axis=1))
pred_classes.replace({0: 'D', 1: 'K'}).to_csv('full_ml.csv', index=False)

In [156]:
pred_classes['class'] = clf.predict_proba(X_test.drop('class', axis=1))[:, 1]
pred_classes.loc[:, 'class'] = pred_classes['class'] > 0.45

/tmp/ipykernel_62673/72656047.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  True False ... False False  True]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  pred_classes.loc[:, 'class'] = pred_classes['class'] > 0.45


In [147]:
pred_classes.loc[:, 'class'].value_counts()

class
False    3907
True     1093
Name: count, dtype: int64

In [148]:
pred_classes.replace({False: 'D', True: 'K'}).to_csv('ml_postfull.csv', index=False)

In [154]:
best = pd.read_csv('best_pred.csv')
best['class'].value_counts()

class
D    3907
K    1093
Name: count, dtype: int64

In [155]:
pred_classes = pred_classes.replace({False: 'D', True: 'K'})
pred_classes['class'].value_counts()

class
D    3907
K    1093
Name: count, dtype: int64